In [1]:
#MAE_test =  metrics.mean_absolute_error(y_test, y_pred)

In [2]:
import sys
!{sys.executable} -m pip install tabulate

In [3]:
#nessicary packages
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from graphviz import Digraph
import graphviz

In [4]:
#get sample data
file = "trip39A.txt"

df = pd.read_csv(file, delimiter = ",")

In [5]:
#drop redundant data
redudantRows = ["DATASOURCE","BASIN","TENDERLOT","SUPPRESSED","JUSTIFICATIONID","LASTUPDATE","NOTE","Unnamed: 0"]

df.drop(columns=redudantRows, inplace=True)
df.shape

(149, 9)

In [6]:
#test target features
df["PLANNEDTIME_TOTAL"] = df["PLANNEDTIME_ARR"] - df["PLANNEDTIME_DEP"]
df["ACTUALTIME_TOTAL"] = df["ACTUALTIME_ARR"] - df["ACTUALTIME_DEP"]
df["DELAY"] = df["ACTUALTIME_TOTAL"] - df["PLANNEDTIME_TOTAL"]

df.shape

(149, 12)

In [7]:
#Drop NA Values
df.dropna(inplace = True)
df.shape

(127, 12)

In [8]:
#format date column correctly
df['DATE'] = np.nan
for i in range(df.shape[0]):
    df['DATE'].iloc[i] = datetime.strptime(df['DAYOFSERVICE'].iloc[i], '%d-%b-%y 00:00:00')
    
#get hour and minutes
df["hours"] = df["ACTUALTIME_DEP"]//3600
df["minutes"] = ((df["ACTUALTIME_DEP"]/3600 - df["ACTUALTIME_DEP"]//3600)*60)//1
df.dtypes

C:\Users\TUDUBLINUSER\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


DAYOFSERVICE          object
TRIPID                 int64
LINEID                object
ROUTEID               object
DIRECTION              int64
PLANNEDTIME_ARR        int64
PLANNEDTIME_DEP        int64
ACTUALTIME_ARR       float64
ACTUALTIME_DEP       float64
PLANNEDTIME_TOTAL      int64
ACTUALTIME_TOTAL     float64
DELAY                float64
DATE                  object
hours                float64
minutes              float64
dtype: object

In [9]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,DATE,hours,minutes
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,2018-02-07 00:00:00,12.0,19.0
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,2018-02-07 00:00:00,9.0,43.0
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,2018-02-07 00:00:00,18.0,30.0
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,2018-06-26 00:00:00,9.0,48.0
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,2018-06-26 00:00:00,14.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,2018-02-18 00:00:00,11.0,40.0
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,2018-02-18 00:00:00,10.0,45.0
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,2018-02-18 00:00:00,10.0,15.0
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,2018-02-18 00:00:00,11.0,15.0


In [10]:
#Get weather data
dfWeather = pd.read_csv("2018_historic_weather_3.txt", delimiter = "\t",  parse_dates=False)
dfWeather.dt = dfWeather.dt.astype('datetime64[ns]')
#dfWeather.dt

dfWeather.dtypes

dt                     datetime64[ns]
dt_iso                         object
timezone                        int64
city_name                      object
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int64
sea_level                     float64
grnd_level                    float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [11]:
#create propper weather columns
dfWeather['dates'] = np.nan

for i in range(8800):
    dfWeather['dates'].iloc[i] = datetime.strptime(dfWeather['dt_iso'].iloc[i], '%Y-%m-%d %H:%M:%S +0000 UTC')

C:\Users\TUDUBLINUSER\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
#select features that may have proise with tarket outcome
dfWeather["DATE"] = dfWeather['dates']
dfweather1 = dfWeather[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity","weather_main"]]

In [13]:
dfweather1.DATE = dfweather1.DATE.astype('datetime64[ns]')
dfweather1.dtypes

C:\Users\TUDUBLINUSER\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


DATE            datetime64[ns]
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
weather_main            object
dtype: object

In [14]:
#create cleaned wether data set
dfWeather_main = pd.get_dummies(dfweather1['weather_main'])
dfWeather_cleaned = pd.concat([dfweather1[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity"]],pd.get_dummies(dfweather1['weather_main'])],axis=1)

dfWeather_cleaned["DATES"] = dfWeather_cleaned["DATE"]
dfWeather_cleaned.DATES = dfWeather_cleaned.DATES.astype('datetime64[ns]')
dfWeather_cleaned.dtypes

DATE          datetime64[ns]
temp                 float64
feels_like           float64
temp_min             float64
temp_max             float64
pressure               int64
humidity               int64
Clear                  uint8
Clouds                 uint8
Drizzle                uint8
Fog                    uint8
Mist                   uint8
Rain                   uint8
Smoke                  uint8
Snow                   uint8
DATES         datetime64[ns]
dtype: object

In [15]:
#create datetime column with all the data
df["month"] = np.nan
df["day"] = np.nan

for i in range(df.shape[0]):
        df["DATE"].iloc[i]= df["DATE"].iloc[i].replace(hour=int(df["hours"].iloc[i]), minute=int(df["minutes"].iloc[i]))
        df["month"].iloc[i] = df.DATE.iloc[i].month

df["month"] = df["month"].astype("int64")        

In [16]:
#df.DATES.astype('object')
df_final = df[["TRIPID","DATE","DIRECTION","PLANNEDTIME_TOTAL","ACTUALTIME_TOTAL","DELAY","LINEID","ROUTEID","hours","month","day"]]
df_final.DATE = df_final.DATE.astype('datetime64[ns]')
df_final.dtypes

TRIPID                        int64
DATE                 datetime64[ns]
DIRECTION                     int64
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                       object
ROUTEID                      object
hours                       float64
month                         int64
day                         float64
dtype: object

# Total Data Set

In [17]:
df_total = pd.merge_asof(df_final.sort_values("DATE"), dfWeather_cleaned, left_on="DATE", right_on="DATE")

In [18]:
for i in range(df_total.shape[0]):
        df_total["day"].iloc[i] = df_total["DATE"].iloc[i].dayofweek       

df_total["day"] = df_total["day"].astype("int64")  

In [19]:
#days of the week:
df_total["weekend"] = np.logical_or(df_total["day"]== 5, df_total["day"]== 6)
df_total["weekend"].replace({False:0,True:1},inplace=True)
df_total["weekday"] = np.logical_or(df_total["day"]>5, df_total["weekend"]== 0)
df_total["weekday"].replace({False:0,True:1},inplace=True)

In [20]:
df_total["DATE"].iloc[1].date()

datetime.date(2018, 2, 7)

In [21]:
#This is to check if the date is a public holiday
# There was no pulblic holiday data in the sample data

ny = pd.Timestamp("2-7-2018")

publicHoliday = [pd.Timestamp("1-1-2018"), pd.Timestamp("3-17-2018"), pd.Timestamp("12-25-2018"), pd.Timestamp("12-26-2018")]

print(ny.date() in publicHoliday)

False


In [22]:
#convert directions to binary column
df_total.DIRECTION.replace(to_replace=2, value=0, inplace=True)

In [23]:
df_total.head()

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Clouds,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,weekend,weekday
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,1,0,0,0,0,0,0,2018-02-07 09:00:00,0,1
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,1,0,0,0,0,0,0,2018-02-07 12:00:00,0,1
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,1,0,0,0,0,0,0,2018-02-07 18:00:00,0,1
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,1,0,0,0,0,0,0,2018-02-18 08:00:00,1,1
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,0,1,0,0,2018-02-18 10:00:00,1,1


In [24]:
#add rush hour
df_total["rush_hour"] = np.logical_or(df_total["hours"]== 7.0, df_total["day"]== 8.0)
df_total["rush_hour"] = np.logical_or(df_total["rush_hour"] == True, df_total["day"]== 16.0)
df_total["rush_hour"] = np.logical_or(df_total["rush_hour"] == True, df_total["day"]== 17.0)
df_total["rush_hour"].replace({False:0,True:1},inplace=True)

In [25]:
df_total_origional = df_total
#df_total_in = df_total[df_total["DIRECTION"]==1]
#df_total_out = df_total[df_total["DIRECTION"]==0]

#df_total = df_total_in

In [26]:
df_total.corr()[["DELAY","ACTUALTIME_TOTAL"]].sort_values(by=['DELAY'], ascending=False)

,DELAY,ACTUALTIME_TOTAL
DELAY,1.000000,0.700393
ACTUALTIME_TOTAL,0.700393,1.000000
DIRECTION,0.263680,0.227464
rush_hour,0.169843,0.082770
day,0.131843,-0.094949
pressure,0.122984,0.117790
Mist,0.111460,-0.001347
hours,0.051162,-0.151248
Drizzle,0.049660,-0.089720
weekend,0.037114,-0.233964


In [27]:
#keep feels like
#drop min_temp, max_temp, temp

In [28]:
df_total.columns

Index(['TRIPID', 'DATE', 'DIRECTION', 'PLANNEDTIME_TOTAL', 'ACTUALTIME_TOTAL',
       'DELAY', 'LINEID', 'ROUTEID', 'hours', 'month', 'day', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'Clear',
       'Clouds', 'Drizzle', 'Fog', 'Mist', 'Rain', 'Smoke', 'Snow', 'DATES',
       'weekend', 'weekday', 'rush_hour'],
      dtype='object')

In [29]:
#Direction, 1 = inbound , 0 = outbound

In [30]:
# Monday = 0
# Tuesday = 1
# Wednesday = 2
# Thursday = 3
# Friday = 4
# Saturday = 5
# Sunday = 6

# What Effects Delay:
- Direction
- Pressure
- Mist
- Rain
- Humidity


# What effects Actualtime_Total:
- Direction
- Pressure
- Hours
- PlannedTime
- Clouds
- Rain
- Humidity
- Month

TRIPID was not included in this as it is a unique identifier for each trip, and there should not be any correlation between the unique identifier for the dataset and the target outcome. 

In [69]:
features = ["DIRECTION","pressure","hours","PLANNEDTIME_TOTAL","Clouds","Rain","humidity","month","weekday", "rush_hour"]

#features = ["DIRECTION","pressure","Mist","Rain","humidity"]

In [70]:
df_total[features]

,DIRECTION,pressure,hours,PLANNEDTIME_TOTAL,Clouds,Rain,humidity,month,weekday,rush_hour
0,0,1024,9.0,4735,1,0,93,2,1,0
1,1,1023,12.0,4876,1,0,87,2,1,0
2,1,1019,18.0,4687,1,0,87,2,1,0
3,0,1017,8.0,3571,1,0,93,2,1,0
4,0,1017,10.0,4163,0,1,93,2,1,0
...,...,...,...,...,...,...,...,...,...,...
122,0,993,9.0,5190,1,0,82,10,1,0
123,0,993,9.0,5190,1,0,82,10,1,0
124,0,993,9.0,5190,1,0,82,10,1,0
125,0,998,18.0,5121,1,0,82,10,1,0


In [71]:
df_total.dtypes

for i in df_total.columns:
    if df_total[i].dtype == "uint8":
        df_total[i] = df_total[i].astype("int64")

In [72]:
df_total.dtypes

TRIPID                        int64
DATE                 datetime64[ns]
DIRECTION                     int64
PLANNEDTIME_TOTAL             int64
ACTUALTIME_TOTAL            float64
DELAY                       float64
LINEID                       object
ROUTEID                      object
hours                       float64
month                         int64
day                           int64
temp                        float64
feels_like                  float64
temp_min                    float64
temp_max                    float64
pressure                      int64
humidity                      int64
Clear                         int64
Clouds                        int64
Drizzle                       int64
Fog                           int64
Mist                          int64
Rain                          int64
Smoke                         int64
Snow                          int64
DATES                datetime64[ns]
weekend                       int64
weekday                     

In [73]:
#df_total["day"] = df_total["DATE"].dayofweek

In [74]:
df_total

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,...,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES,weekend,weekday,rush_hour
0,6261203,2018-02-07 09:43:00,0,4735,5265.0,530.0,39A,39A_43,9.0,2,...,0,0,0,0,0,0,2018-02-07 09:00:00,0,1,0
1,6246028,2018-02-07 12:19:00,1,4876,5656.0,780.0,39A,39A_40,12.0,2,...,0,0,0,0,0,0,2018-02-07 12:00:00,0,1,0
2,6251013,2018-02-07 18:30:00,1,4687,5932.0,1245.0,39A,39A_40,18.0,2,...,0,0,0,0,0,0,2018-02-07 18:00:00,0,1,0
3,6274915,2018-02-18 08:19:00,0,3571,3920.0,349.0,39A,39A_43,8.0,2,...,0,0,0,0,0,0,2018-02-18 08:00:00,1,1,0
4,6271772,2018-02-18 10:15:00,0,4163,4091.0,-72.0,39A,39A_43,10.0,2,...,0,0,0,1,0,0,2018-02-18 10:00:00,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8024541,2018-10-11 09:19:00,0,5190,5557.0,367.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
123,8025109,2018-10-11 09:29:00,0,5190,5587.0,397.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
124,8020475,2018-10-11 09:51:00,0,5190,5419.0,229.0,39A,39A_43,9.0,10,...,0,0,0,0,0,0,2018-10-11 09:00:00,0,1,0
125,8023269,2018-10-11 18:21:00,0,5121,6079.0,958.0,39A,39A_43,18.0,10,...,0,0,0,0,0,0,2018-10-11 18:00:00,0,1,0


In [75]:
#x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["significant_delay"], test_size=0.3, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(df_total[features], df_total["ACTUALTIME_TOTAL"], test_size=0.3, random_state=0)

In [76]:
# building the models
# Model 1
logreg = LogisticRegression().fit(x_train[features], y_train)

C:\Users\TUDUBLINUSER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [77]:
#logistic regression results
logreg_table = {"Features":features, "Coefficient": logreg.coef_[0]}
print("\nIntercept: \n", logreg.intercept_,"\n")

print(tabulate(logreg_table, headers='keys'))


Intercept: 
 [ 2.26363868e-03  9.54033320e-04  9.54033320e-04  8.80627315e-04
  3.33461515e-04  2.47364786e-04  3.36445247e-04  4.24979130e-04
  7.92584829e-04  5.66169201e-04  3.36445247e-04 -2.68740386e-04
  1.71553985e-04  4.24979130e-04 -2.68740386e-04  3.36445247e-04
  6.76780287e-05 -2.68740386e-04  2.19371906e-04  3.87311661e-04
  2.48647669e-04  2.47364786e-04  2.48647669e-04  2.85285796e-04
  3.92533810e-04  3.79431151e-04  6.93789407e-05  9.33395068e-05
 -5.68710766e-04 -2.68757123e-04 -1.82796789e-04 -3.50272580e-04
  3.85097704e-04 -2.68757123e-04 -2.68740386e-04  5.49233753e-05
  3.54888288e-05  2.93782891e-04  1.71553985e-04 -3.11502211e-04
  6.93789407e-05  2.55991997e-04 -5.11577402e-05  2.51283521e-04
  3.36526433e-04  1.71439451e-04 -2.68740386e-04  5.49233753e-05
  7.97787139e-05 -1.54837415e-04  3.79431151e-04  9.34737623e-05
 -7.40237107e-04 -5.45991934e-04 -2.68757123e-04  9.34737623e-05
  2.56007401e-06 -5.68085687e-04  1.71553985e-04  1.71439451e-04
 -2.6875712

In [78]:
#train predictions
logreg_predictions_train = (logreg.predict(x_train[features]))
MAE_train_log =  metrics.mean_absolute_error(y_train, logreg_predictions_train)
MSE_train_log = metrics.mean_squared_error(y_train, logreg_predictions_train)
RMSE_train_log = MSE_train_log**0.5

print("Train MAE:", MAE_train_log)
print("Train MSE:", MSE_train_log)
print("Train RMSE:", RMSE_train_log)

Train MAE: 300.1818181818182
Train MSE: 270573.38636363635
Train RMSE: 520.1666909401604


In [79]:
#test predictions
logreg_predictions_test = (logreg.predict(x_test[features]))
MAE_test_log =  metrics.mean_absolute_error(y_test, logreg_predictions_test)
MSE_test_log = metrics.mean_squared_error(y_test, logreg_predictions_test)
RMSE_test_log = MSE_test_log**0.5

In [80]:
MAE_test_log
print("Test MAE:", MAE_test_log)
print("Test MSE:", MSE_test_log)
print("Test RMSE:", RMSE_test_log)

Test MAE: 578.3589743589744
Test MSE: 573477.8974358974
Test RMSE: 757.2832346195823


# Decision Trees

In [43]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=0)
rfc.fit(x_train[features], y_train)

RandomForestClassifier(oob_score=True, random_state=0)

In [51]:
# build random forests on training set 
dtc2 = DecisionTreeClassifier(max_depth=2, random_state=0) 
dtc4 = DecisionTreeClassifier(max_depth=4, random_state=0) 
dtc10 = DecisionTreeClassifier(max_depth=10, random_state=0) 

dtc2.fit(x_train[features], y_train) 
dtc4.fit(x_train[features], y_train) 
dtc10.fit(x_train[features], y_train) 

print("Max depth 2: \n",dtc2) 
print("Max depth 4: \n",dtc4) 
print("Max depth 10: \n",dtc10)

Max depth 2: 
 DecisionTreeClassifier(max_depth=2, random_state=0)
Max depth 4: 
 DecisionTreeClassifier(max_depth=4, random_state=0)
Max depth 10: 
 DecisionTreeClassifier(max_depth=10, random_state=0)


In [53]:
#obtain graphical representation of decision trees
#NOTE: depending on how graphviz is set up on your computer, this operation might not work, so I have included a copy of 
#the trees as pngs
with open("DecisionTree2.dot", 'w') as f0:
    f1 = export_graphviz(dtc2, out_file=f0, feature_names=x_train[features].columns)
with open("DecisionTree4.dot", 'w') as f1:
    f1 = export_graphviz(dtc4, out_file=f1, feature_names=x_train[features].columns)
with open("DecisionTree10.dot", 'w') as f2:
    f2 = export_graphviz(dtc10, out_file=f2, feature_names=x_train[features].columns)
!dot -Tpng DecisionTree2.dot -o DecisionTree2.png
!dot -Tpng DecisionTree4.dot -o DecisionTree4.png
!dot -Tpng DecisionTree10.dot -o DecisionTree10.png

<img src="DecisionTree2.png">

<img src="DecisionTree4.png">

<img src="DecisionTree10.png">

In [59]:
rf_predictions_train = rfc.predict(x_train[features])

#train results
MAE_train_rf =  metrics.mean_absolute_error(y_train, rf_predictions_train)
MSE_train_rf = metrics.mean_squared_error(y_train, rf_predictions_train)
RMSE_train_rf = MSE_train_rf**0.5

In [61]:
print("Train MAE:", MAE_train_rf)
print("Train MSE:", MSE_train_rf)
print("Train RMSE:", RMSE_train_rf)

Train MAE: 177.17045454545453
Train MSE: 135773.48863636365
Train RMSE: 368.4745427249536


In [63]:
rf_predictions_test = (rfc.predict(x_test[features]))
MAE_test_rf =  metrics.mean_absolute_error(y_test, rf_predictions_test)
MSE_test_rf = metrics.mean_squared_error(y_test, rf_predictions_test)
RMSE_test_rf = MSE_test_rf**0.5

In [64]:
print("Test MAE:", MAE_test_rf)
print("Test MSE:", MSE_test_rf)
print("Test RMSE:", RMSE_test_rf)

Test MAE: 561.6410256410256
Test MSE: 506512.8205128205
Test RMSE: 711.6971410036859


In [81]:
importance = pd.DataFrame({'feature': x_train[features].columns, 'importance':rfc.feature_importances_})
importance.sort_values('importance', ascending=False)

,feature,importance
2,hours,0.244746
3,PLANNEDTIME_TOTAL,0.187216
1,pressure,0.164933
6,humidity,0.161628
7,month,0.100823
0,DIRECTION,0.056393
4,Clouds,0.041532
9,rush_hour,0.024069
5,Rain,0.018660
8,weekday,0.000000


# Linear Model

In [44]:
linreg = LinearRegression().fit(x_train[features], y_train)

In [45]:
linreg_table = {"Features":features, "Coefficient": linreg.coef_}
print("\nIntercept: \n", linreg.intercept_,"\n")

print(tabulate(linreg_table, headers='keys'))


Intercept: 
 -6182.009008783935 

Features             Coefficient
-----------------  -------------
DIRECTION               84.2231
pressure                 6.38556
hours                   13.1826
PLANNEDTIME_TOTAL        1.19198
Clouds                 -80.6801
Rain                  -255.955
humidity                -6.20104
month                  -52.4342
weekday                  0
rush_hour              224.533


In [46]:
linreg_predictions_train = (linreg.predict(x_train[features]))
MAE_train_lin =  metrics.mean_absolute_error(y_train, linreg_predictions_train)
MSE_train_lin = metrics.mean_squared_error(y_train, linreg_predictions_train)
RMSE_train_lin = MSE_train_lin**0.5

print("Train MAE:", MAE_train_lin)
print("Train MSE:", MSE_train_lin)
print("Train RMSE:", RMSE_train_lin)

Train MAE: 384.2604437988877
Train MSE: 251941.63769285
Train RMSE: 501.9378823050219


In [47]:
linreg_predictions_test = (linreg.predict(x_test[features]))
MAE_test_lin =  metrics.mean_absolute_error(y_test, linreg_predictions_test)
MSE_test_lin = metrics.mean_squared_error(y_test, linreg_predictions_test)
RMSE_test_lin = MSE_test_lin**0.5

In [48]:
print("Test MAE:", MAE_test_lin)
print("Test MSE:", MSE_test_lin)
print("Test RMSE:", RMSE_test_lin)

Test MAE: 408.8298439056371
Test MSE: 282698.0210078352
Test RMSE: 531.6935404985048
